# Verificação dos dados oferta do artesanato paraibano 2022

```Autor: Flávio Macaúbas Torres Filho```

In [1]:
# Pacote necessários

import pandas as pd
from os.path import join # Cria paths 
from unidecode import unidecode # Retirar acentos
from datetime import datetime # Manipulação de datas

# Manipulações matemáticas
import math 
import numpy as np


# Path dos arquivos
path_limpos = "C:\\Users\\Macaubas\\Desktop\\Python\\Labimec\\Artesanato\\Dados\\Limpos"
path_originais = "C:\\Users\\Macaubas\\Desktop\\Python\\Labimec\\Artesanato\\Dados\\Originais"

## Oferta Artesanato Paraíba 2022

In [406]:
# Leitura do arquivo
arquivo = join(path_orginais, "oferta_artesanal_pb_2022.xlsx")
oferta = pd.read_excel(arquivo)

# Ajuste dos nomes das colunas
novas_colunas = oferta.columns
novas_colunas = [col.strip().replace('.','_').replace('?','').replace('/','').replace(':','') for col in novas_colunas] # Retirando caracteres indesejados
novas_colunas = [col.strip().replace(' ','_') for col in novas_colunas] # substituindo espaços em branco por _
novas_colunas = [unidecode(col.upper()) for col in novas_colunas] # colocando maiúsculo e retirando acentos
novas_colunas[-1] = 'ENTREVISTADOR' # ajustando nome da última coluna

oferta.columns = novas_colunas

#### Convertendo tipagem ----

## Convertendo tipos das colunas para string --
colunas_para_str = ['1_3_ONDE_MOROU_ANTERIORMENTE',
                    'ESTADO_ONDE_MOROU',
                    '1_4_ONDE_NASCEU',
                    '1_5_EM_QUE_ANO',
                    '1_8_SE_A_RESPOSTA_ACIMA_FOR_SIM,_QUAL_FOI_O_CURSO',
                    '1_11_1_DESCREVA_O_PRINCIPAL_PRODUTO_(REFERENTE_AO_ITEM_ANTERIOR)',
                    '1_13_PREFERE_UTILIZAR_EM_SEUS_TRABALHOS_REFERENCIAS',
                    '2_4_SE_A_RESPOSTA_ACIMA_FOR_SIM,_QUAL_MEMBRO_TRABALHA_COM_ARTESANATO',
                    '2_6_SE_A_RESPOSTA_ACIMA_FOR_SIM,_QUAL_ASSOCIACAO_OU_COOPERATIVA',
                    '2_10_SE_A_RESPOSTA_ACIMA_FOR_SIM,_QUANTAS_PESSOAS_VOCE_JA_ENSINOU',
                    '2_12_SE_A_RESPOSTA_ACIMA_FOR_SIM,_QUAL_O_TEMA_TEM_INTERESSE',
                    '2_13_PARTICIPA_ATIVAMENTE_DE_QUAIS_REDES_SOCIAIS',
                    '2_15_TELEFONE_PARA_CONTATO',
                    '3_16_SE_A_RESPOSTA_ACIMA_FOR_SIM,_QUAL_O_CNPJ_RAZAO_SOCIAL',
                    '3_17_VOCE_PRETENDE',
                    '3_22_GOSTARIA_DE_RELATAR_ALGUMA_DIFICULDADE_OU_PERSPECTIVA_QUE_NAO_FORAM_CONTEMPLADAS_NA_PESQUISA'         
                   ]

for col in colunas_para_str:
    oferta[col] = oferta[col].astype(str)
    
## Colocando todas as colunas strings em maiúsculo --
for i, value in enumerate((oferta.applymap(type) == str).all(0)):
    if value == True:
        nome_coluna = (oferta.applymap(type) == str).all(0).index[i]
        oferta[nome_coluna] = oferta[nome_coluna].apply(lambda x: x.strip().upper())
    

## Convertendo coluna 1.5 para datetime --
oferta['1_5_EM_QUE_ANO'] = pd.to_datetime(oferta['1_5_EM_QUE_ANO'])

## Substituindo observações ausentes pela diferença do ano atual da Coluna 1.5 --
anos = list((datetime.now() - oferta['1_5_EM_QUE_ANO'])) # Calcula diferença em dias e hora
anos = [a.days for a in anos] # Extrai apenas o dia
anos = [int(a/365) for a in anos] # Encontra o valor em anos e arredonda para baixo
anos

novos_anos = []

for i in range(len(oferta['1_2_HA_QUANTO_TEMPO'].values)):
    if math.isnan(oferta['1_2_HA_QUANTO_TEMPO'][i]):
        novos_anos.append(anos[i])
    else:
        novos_anos.append(int(oferta['1_2_HA_QUANTO_TEMPO'][i]))
        
# Atualiza valores
oferta['1_2_HA_QUANTO_TEMPO'] = novos_anos

## Criando campo de idade --
oferta['IDADE'] = anos

## Ajustando colunas 1.3 e "Estado onde nasceu", removendo os valores no caso de serem iguais as respostas de  1.1 --
oferta.loc[oferta['1_1_CIDADE_ONDE_VIVE']==oferta['1_3_ONDE_MOROU_ANTERIORMENTE'], '1_3_ONDE_MOROU_ANTERIORMENTE'] = None
oferta.loc[oferta['1_1_CIDADE_ONDE_VIVE']==oferta['1_3_ONDE_MOROU_ANTERIORMENTE'], 'ESTADO_ONDE_NASCEU'] = None

## Ajustando coluna 1.7 por inconsistência de respostas - Deve ser só 'SIM' ou 'NÃO' --
oferta.loc[oferta['1_7_JA_FEZ_ALGUM_CURSO_DE_ARTES'] == 'CURSO DE BIOJOIAS - IFPB', '1_7_JA_FEZ_ALGUM_CURSO_DE_ARTES'] = 'SIM'

## Ajustando coluna 1.5 para formato telefônico --
oferta['2_15_TELEFONE_PARA_CONTATO'] = oferta['2_15_TELEFONE_PARA_CONTATO'].apply(lambda x: x.strip().replace('(','').replace(')','').replace(' ','').replace('-',''))
oferta['2_15_TELEFONE_PARA_CONTATO'] = oferta['2_15_TELEFONE_PARA_CONTATO'].apply(lambda x: '83'+ x if len(x) == 9  else x)

## Interpolando valor ausente da coluna 3.9 pela média de peças produzidas (APENAS 1 OBSERVAÇÃO) --
oferta['3_9_QUANTAS_PECAS_CONSEGUE_PRODUZIR_POR_MES'] = oferta['3_9_QUANTAS_PECAS_CONSEGUE_PRODUZIR_POR_MES'].replace(np.nan, media_pecas_produzidas)

## Se a resposta da coluna 3.15 foi não, a coluna 3.16 não deve ter resposta --
oferta.loc[oferta['3_15_POSSUI_CNPJ__RAZAO_SOCIAL']== 'NÃO', '3_16_SE_A_RESPOSTA_ACIMA_FOR_SIM,_QUAL_O_CNPJ_RAZAO_SOCIAL'] = np.nan



#### Salvando arquivo ----
oferta.to_csv(join(path_limpos, 'oferta_artesanal_pb_2022_limpo.csv'), index=False)

## Demanda LOCAL - Artesnato PB

In [37]:
# Leitura do arquivo
arquivo = join(path_originais, "VISITANTE LOCAL (respostas).xlsx")
demanda_local = pd.read_excel(arquivo)

# Ajuste dos nomes das colunas
novas_colunas = demanda_local.columns
novas_colunas = [col.strip().replace('.','_').replace('?','').replace('/','').replace(':','').replace('\t','') for col in novas_colunas] # Retirando caracteres indesejados
novas_colunas = [col.strip().replace(' ','_') for col in novas_colunas] # substituindo espaços em branco por _
novas_colunas = [unidecode(col.upper()) for col in novas_colunas] # colocando maiúsculo e retirando acentos


demanda_local.columns = novas_colunas

## Convertendo tipos das colunas para string --
COLUNAS_NAO_STR = ['DATA',
                   '5__HA_QUANTO_ANOS_EM_JOAO_PESSOA',
                   '14__QUANTAS_PECAS_DE_ARTESANATO_VOCE_ESTIMA_TER_EM_SUA_CASA',
                   '15_QUANTAS_SAO_DO_ARTESANATO_PARAIBANO']

colunas_para_str = [col for col in novas_colunas if col not in COLUNAS_NAO_STR]

for col in colunas_para_str:
    demanda_local[col] = demanda_local[col].astype(str)

## Colocando todas as colunas strings em maiúsculo --
for i, value in enumerate((demanda_local.applymap(type) == str).all(0)):
    if value == True:
        nome_coluna = (demanda_local.applymap(type) == str).all(0).index[i]
        demanda_local[nome_coluna] = demanda_local[nome_coluna].apply(lambda x: x.strip().upper())
        
## Ajuste nome dos bairros para padrão do pacote geobr do R
demanda_local['2__BAIRRO_ONDE_MORA'] = np.select(
    [
        demanda_local['2__BAIRRO_ONDE_MORA'] == 'ALTIPLANO',
        demanda_local['2__BAIRRO_ONDE_MORA'] == 'CRISTO',
        demanda_local['2__BAIRRO_ONDE_MORA'] == 'BAIRRO DOS ESTADOS',
        demanda_local['2__BAIRRO_ONDE_MORA'] == 'NOVA MANGABEIRA',
        demanda_local['2__BAIRRO_ONDE_MORA'] == 'BAIRRO DOS IPÊS'
    ],
    [
        'ALTIPLANO CABO BRANCO',
        'CRISTO REDENTOR',
        'ESTADOS',
        'MANGABEIRA',
        'IPÊS'
    ],
    default = demanda_local['2__BAIRRO_ONDE_MORA']
)

# Retirando acentuação dos bairros
demanda_local['2__BAIRRO_ONDE_MORA'] = demanda_local['2__BAIRRO_ONDE_MORA'].apply(lambda x: unidecode(x))
    

#### Salvando arquivo ----
demanda_local.to_csv(join(path_limpos, 'DEMANDA_LOCAL_LIMPO.csv'), index=False)

# Demanda Turistas - Artesanato PB

In [14]:
# Leitura do arquivo
arquivo = join(path_originais, "TURISTA (respostas).xlsx")
demanda_turista = pd.read_excel(arquivo)

# Ajuste dos nomes das colunas
novas_colunas = demanda_turista.columns
novas_colunas = [col.strip().replace('.','_').replace('?','').replace('/','').replace(':','').replace('\t','') for col in novas_colunas] # Retirando caracteres indesejados
novas_colunas = [col.strip().replace(' ','_') for col in novas_colunas] # substituindo espaços em branco por _
novas_colunas = [unidecode(col.upper()) for col in novas_colunas] # colocando maiúsculo e retirando acentos


demanda_turista.columns = novas_colunas

## Convertendo tipos das colunas para string --
COLUNAS_NAO_STR = ['DATA',
                   '1_5_QUANTOS_DIAS_FICARA_EM_JOAO_PESSOA',
                   '1_9_QUANTO_COMPROU_DE_ARTESANATO_NESSA_VIAGEM',
                   '1_15_QUANTAS_PECAS_DE_ARTESANATO_VOCE_ESTIMA_TER_EM_SUA_CASA',
                   '1_16_QUANTAS_SAO_DO_ARTESANATO_PARAIBANO']

colunas_para_str = [col for col in novas_colunas if col not in COLUNAS_NAO_STR]

for col in colunas_para_str:
    demanda_turista[col] = demanda_turista[col].astype(str)
    
## Colocando todas as colunas strings em maiúsculo --
for i, value in enumerate((demanda_turista.applymap(type) == str).all(0)):
    if value == True:
        nome_coluna = (demanda_turista.applymap(type) == str).all(0).index[i]
        demanda_turista[nome_coluna] = demanda_turista[nome_coluna].apply(lambda x: x.strip().upper())
        
        

#### Salvando arquivo ----
demanda_turista.to_csv(join(path_limpos, 'DEMANDA_TURISTA_LIMPO.csv'), index=False)
